<a href="https://colab.research.google.com/github/Iispar/hlt-project/blob/main/course_project_2023_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to HLT 2023 Project (Template)

- Student(s) Name(s): Iiro Partanen
- Date: -
- Chosen Corpus: emotion
- Contributions (if group project):

### Corpus information

- Description of the chosen corpus: 
Emotion is a dataset of English Twitter messages with six basic emotions: anger, fear, joy, love, sadness, and surprise. For more detailed information please refer to the paper.
- Paper(s) and other published materials related to the corpus: 
  - CARER: Contextualized Affect Representations for Emotion Recognition (Saravia et al., EMNLP 2018)
  - https://paperswithcode.com/dataset/emotion
- State-of-the-art performance (best published results) on this corpus: 95% f1

---

## 1. Setup

In [7]:
!pip3 install -q transformers datasets evaluate
import datasets
import sklearn.feature_extraction

---

## 2. Data download and preprocessing

### 2.1. Download the corpus

In [11]:
dset=datasets.load_dataset("emotion");

  0%|          | 0/3 [00:00<?, ?it/s]

### 2.2. Preprocessing

In [30]:
# vectorizes one item
def vectorize_item(item):
    vectorized = vectorizer.transform([item["text"]]);
    non_zero_features = vectorized.nonzero()[1]; # removes the zeros and we take we take columns from .nonzero
    non_zero_features += 1; # index zero is for padding so let's avoid it by adding 1 to all.

    return {"input_ids":non_zero_features} 

In [36]:
dset.shuffle(); #shuffle dataset for safety

# vectorization.
vectorizer = sklearn.feature_extraction.text.CountVectorizer(binary=True,max_features=20000); # get the vectorizer. Selected 20k to start with.
texts=[ex["text"] for ex in dset["train"]] # get all texts from train
vectorizer.fit(texts) # fitting the vectorizer

# vectorize the whole dataset.
dset_tokenized = dset.map(vectorize_item,num_proc=4)

---

## 3. Machine learning model

### 3.1. Model training

In [ ]:
# Your code to train the machine learning model on the training set and evaluate the performance on the validation set here

### 3.2 Hyperparameter optimization

In [ ]:
# Your code for hyperparameter optimization here

### 3.3. Evaluation on test set

In [ ]:
# Your code to evaluate the final model on the test set here

---

## 4. Results and summary

### 4.1 Corpus insights

(Briefly discuss what you learned about the corpus and its annotation)

The corpus includes Twitter messages in english and they have been annotated with six basic emotions which are anger, fear, joy, love, sadness, and surprise. 

By the paper the tweets have been selected with some hashtags and then annotated. The selected hastags can be seen from the paper.


### 4.2 Results

(Briefly summarize your results)

### 4.3 Relation to state of the art

(Compare your results to the state-of-the-art performance)

---

## 5. Bonus Task (optional)

### 5.1. Annotating out-of-domain documents

(Briefly describe the chosen out-of-domain documents)

(Briefly describe the process of annotation)

### 5.2 Conversion into dataset

In [ ]:
# Your code to convert the annotations into a dataset here

### 5.3. Model evaluation on out-of-domain test set

In [ ]:
# Your code to evaluate the model on the out-of-domain test set here

### 5.4 Bonus task results

(Present the results of the evaluation on the out-of-domain test set)

### 5.5. Annotated data

In [ ]:
# Include your annotated out-of-domain data here